In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
else:
    try:
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin

In [ ]:
cpp_code = """
    #include <pybind11/pybind11.h>
    #include <dolfin/log/log.h>

    void throw_dolfin_error()
    {
        dolfin::dolfin_error("JITed module", "not throw an error", "On purpose");
    }

    void throw_runtime_error()
    {
        throw std::runtime_error("My runtime error");
    }

    PYBIND11_MODULE(SIGNATURE, m)
    {
        m.def("throw_dolfin_error", &throw_dolfin_error);
        m.def("throw_runtime_error", &throw_runtime_error);
    }
"""
cpp_module = dolfin.compile_cpp_code(cpp_code)

In [ ]:
try:
    cpp_module.throw_runtime_error()
except RuntimeError as e:
    assert str(e) == "My runtime error"
else:
    raise RuntimeError("An error should have been raised")

In [ ]:
try:
    cpp_module.throw_dolfin_error()
except RuntimeError as e:
    assert "Unable to not throw an error" in str(e)
else:
    raise RuntimeError("An error should have been raised")